In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import unicodedata as ud
import re
text = requests.get('http://www.ttnews.com/fuel/indexf.aspx')
text = text.text
text = BeautifulSoup(text)

/Users/satishjoshi/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [2]:
header = text.find_all(lambda Tag: Tag.name=='td' and Tag.get('bgcolor') =='#c0c0c0')
top_header_lst = []
for w in header:
    t = w.get_text()
    t = ud.normalize('NFKD', t).encode('ascii', 'ignore')
    top_header_lst.append(t)

In [3]:
all_links = text.select('table tr table tr td a')
links = []
link_texts = []
for w in all_links:
    if (re.search('\/fuel\/', str(w)) != None) and (re.search('\"_blank\"', str(w)) != None):
        t = w.get('href')
        #bad links from the page itself :(
        t = t.replace(".asp.htm", ".asp")        
        links.append(t)
        t = w.get_text()
        t = ud.normalize('NFKD', t).encode('ascii', 'ignore')
        link_texts.append(t)

In [4]:
all_labels_links = text.select('table tr table tr td td')
years = []
for w in all_labels_links:
    if (re.search('<td>', str(w)) != None) and (re.search('2[0-9][0-9][0-9]', str(w)) != None) and \
    (re.search('\/fuel\/', str(w)) == None):
        t = w.get_text()
        t = ud.normalize('NFKD', t).encode('ascii', 'ignore')
        years.append(t)

In [5]:
price_table_columns = ['Index']+top_header_lst + ['DOELink','COMDataLink','Updated']
price_table = pd.DataFrame(data=np.zeros((0,len(price_table_columns))), columns=price_table_columns)

data_dict = {}
i = 0
for index,item in enumerate(years):
    data_dict['Index'] = index
    data_dict[top_header_lst[0]] = item
    data_dict[top_header_lst[1]] = link_texts[index*2]
    data_dict[top_header_lst[2]] = link_texts[index*2+1]
    data_dict['DOELink'] = links[index*2]
    data_dict['COMDataLink'] = links[index*2+1]
    data_dict['Updated'] = 'No'

    single = pd.DataFrame([data_dict], columns=price_table_columns)
    price_table = price_table.append(single)
price_table.to_csv('price_table.csv')

In [6]:
def get_header(data, lst, tag_color = 1):
#    lst = []
#    header_row = soup.find('bgColor'= "#ffffcc")
    if tag_color == 1:
        header_row = data.find_all(lambda Tag: Tag.name=='tr' and Tag.get('bgcolor') \
                               in {'#ffffcc',"#FFFFCC"})
    else:
        header_row = data.find_all(lambda Tag: Tag.name=='tr'  and Tag.get('b'))
#    print header_row
    for row in header_row:
#        print row
        col = row.findAll('td')
#        print col
        for td in col:
#            print td
            t = td.get_text().rstrip("\r\n ")
            t = t.replace("\r\n                                  ", "")
            t = t.replace(" \r\n      ", " ")
            t = t.replace("\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t","")
            t = t.replace(" \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t", " ")
            t = ud.normalize('NFKD', t).encode('ascii', 'ignore')   
#            print "Value is %s" %t
            lst.append(t)
#print header_list

#    return lst
    return

def row_colors(Tag):
    return Tag.get('bgcolor') in {'#CCCCCC','#FFFFFF','cccccc','#ffffff''#cccccc'}

def modify_values(value):
    if re.search('/', value)==None:
        if re.search('\.$', value):
            return float(value[:-1])  
        else:
            return float(value) 
    else:
        return value

def get_prices(data, lst):
#    lst = []
    t = data.find_all(lambda Tag: Tag.name=='tr' and row_colors(Tag))
#    print "I am in get prices\n"
#    print t
    for w in t:
        value = w.get_text().strip('\n ')
#        print value
        if value.find('/') != -1 :
            date = ud.normalize('NFKD', value).encode('ascii', 'ignore')
#        print date
        lst.append(map(lambda x: modify_values(x),date.split()))
#    return lst
    return
def get_doe_yearly_prices(data, header, prices, tag_color = 1):
#print type(yearly_fuel_price_DOE)
#    data = data.text
#print type(yearly_fuel_price_DOE)
#    data = BeautifulSoup(data)
#print yearly_fuel_price_DOE.prettify()

#header_list = []
#price_list = []
    get_header(data, header, tag_color)
#    print "Get header is done\n"
    get_prices(data, prices)
#    print "Get prices is done\n"
    return  

In [7]:
def get_yearly_doe_prices(link, header, prices, tag_color = 1): 
    yearly_fuel_price_DOE = requests.get(link)
    yearly_fuel_price_DOE = yearly_fuel_price_DOE.text
    yearly_fuel_price_DOE = BeautifulSoup(yearly_fuel_price_DOE)
    get_doe_yearly_prices(yearly_fuel_price_DOE, header, prices, tag_color)


print type(price_table)
DOE_header_columns = ['Year', 'Link', 'Date', 'U.S. Average', 'East Coast', \
    'New England', 'Central Atlantic', 'Lower Atlantic', 'Midwest', 'Gulf Coast', \
    'Rocky Mountain', 'West Coast', 'California']

DOE_header = pd.DataFrame(data=np.zeros((0,len(DOE_header_columns))), columns=DOE_header_columns)
DOE_values = pd.DataFrame(data=np.zeros((0,len(DOE_header_columns))), columns=DOE_header_columns)

for index, row in price_table.iterrows():
    header_list2 = []
    price_list2 = []
    if row['Year'] in  ['2000', '2001', '2002']:
        get_yearly_doe_prices (row["DOELink"], header_list2, price_list2, 0)
    else:
        get_yearly_doe_prices (row["DOELink"], header_list2, price_list2)
    header_dict = {}
    value_dict = {}
    header_dict['Year'] = row['Year']
    header_dict['Link'] = row["DOELink"]
    value_dict['Year'] = row['Year']
    value_dict['Link'] = row["DOELink"]
    for index, region in enumerate(header_list2):
        header_dict[DOE_header_columns[index+2]] = region
    single = pd.DataFrame([header_dict], columns=DOE_header_columns)
    DOE_header = DOE_header.append(single)
    for index, value_lst in enumerate(price_list2):
        for index, value in enumerate(value_lst):
            value_dict[DOE_header_columns[index+2]] = value
        one_week = pd.DataFrame([value_dict], columns=DOE_header_columns)
        DOE_values = DOE_values.append(one_week)
DOE_header.to_csv('DOE_header.csv')
DOE_values.to_csv('DOE_values.csv')    

<class 'pandas.core.frame.DataFrame'>
